In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import datetime
import time

import pandas as pd
import numpy as np

In [2]:
import multiprocessing
from tqdm.notebook import tqdm

In [3]:
web = './chromedriver'

In [4]:
numdays = 2

In [5]:
category_id_list = [str(i) for i in range(100,106)]
news_number_list = [str(i) for i in range(1,31)]

In [6]:
base = datetime.datetime.today()
date_list = [(base - datetime.timedelta(days=x)).strftime('%Y%m%d') for x in range(numdays)]

In [7]:
def crawling_news_data(category_id):
    data_list = []
    driver = webdriver.Chrome(web)
    for date_string in date_list:
        for number in news_number_list:
            _url = "https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&sectionId="+category_id+"&date="+date_string
            driver.get(_url)

            driver.find_element_by_xpath('//*[@id="wrap"]/table/tbody/tr/td[2]/div/div[4]/ol/li['+number+']/div/div[1]/a').click()
            title = driver.find_element_by_xpath('//*[@id="articleTitle"]').text
            date = driver.find_element_by_xpath('//*[@id="main_content"]/div[1]/div[3]/div/span[1]').text
            contents = driver.find_element_by_xpath('//*[@id="articleBodyContents"]').text
            current_url = driver.current_url
            driver.find_element_by_xpath('//*[@id="main_content"]/div[1]/div[3]/div/div[3]/div[2]/div[1]').click()
            time.sleep(2)
            try:
                sum_bot = driver.find_element_by_xpath('//*[@id="main_content"]/div[1]/div[3]/div/div[3]/div[2]/div[1]/div/div[2]/div[1]').text
            except:
                sum_bot = None
                
            df = pd.DataFrame([title, date, category_id, contents, sum_bot, current_url]).T
            df.columns = ['title', 'date', 'category_id', 'contents', 'summarisation', 'url']
            data_list.append(df)
            driver.back()
    driver.close()
    all_data = pd.concat(data_list)
    return all_data

In [8]:
num_cpu = multiprocessing.cpu_count() - 1
start_time = datetime.datetime.now()

print("START DATE :", start_time)


pool = multiprocessing.Pool(num_cpu)
results = pool.imap(crawling_news_data, category_id_list)

pool.close()
pool.join()

end_time = datetime.datetime.now()
print("END DATE :", end_time)

START DATE : 2020-09-17 16:38:50.384438
END DATE : 2020-09-17 16:43:33.497394


In [9]:
all_data = list(results)
all_data = pd.concat(all_data).reset_index(drop=True)

In [10]:
all_data.to_csv('news_data.csv', index=False)